# Dataset creation to train a YoloV8 model for tank detection

This notebook shows how to build a dataset of annotated images to train a computer vision model for object detection. We use images from open source object-detection datasets to create a dataset of military vehicles and format it correctly for YoloV8 training.

We use [fiftyone](https://github.com/voxel51/fiftyone) to convert, merge, label and format the images prior to training with [Yolov8](https://github.com/ultralytics/ultralytics).

### Defining classes

In order to have a model that discriminates between different types of military vehicles, we'll try to define large class labels by using Wikipedia's [Military vehicles by type](https://en.wikipedia.org/wiki/Category:Military_vehicles_by_type) category.

We'll use 4 classes : 

- **Armoured fighting vehicle (AFV)** is an armed combat vehicle protected by armour, generally combining operational mobility with offensive and defensive capabilities. AFVs can be wheeled or tracked. Examples of AFVs are tanks, armoured cars, assault guns, self-propelled guns, infantry fighting vehicles (IFV), and armoured personnel carriers (APC).
- **Armoured personnel carrier (APC)** is a broad type of armoured military vehicle designed to transport personnel and equipment in combat zones.
- **Military engineering vehicle (MEV)** is a vehicle built for construction work or for the transportation of combat engineers on the battlefield.
- **Light armoured vehicle (LAV) (incl. Reconnaissance vehicle - RV)** is the lightest weight class military vehicle category. A Jeep-like four-wheel drive vehicle for military use, with light or no armour. **Reconnaissance vehicle (RV)** is a military vehicle used for forward reconnaissance. Both tracked and wheeled reconnaissance vehicles are in service.

### Setup

We start by setting up some logging.

In [1]:
import logging

logger = logging.getLogger('adomvi')
logger.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(ch)

### Download images from ImageNet

The first dataset we'll use is ImageNet21k. The ImageNet21k dataset is available at [https://image-net.org/download-images.php](https://image-net.org/download-images.php). You need to register and be granted access to download the images. We use the Winter 21 version since it gives the option of downloading the images for a single synset: https://image-net.org/data/winter21_whole/SYNSET_ID.tar, e.g., https://image-net.org/data/winter21_whole/n02352591.tar. The processed version of ImageNet21k is available here : https://github.com/Alibaba-MIIL/ImageNet21K. The class ids and names are available here https://github.com/google-research/big_transfer/issues/7#issuecomment-640048775.

We'll begin by downloading the class names that are in ImageNet21k and look for relevant classes that we can use.

In [2]:
from pathlib import Path

imagenet_dir = Path() / "imagenet"

In [3]:
from adomvi.datasets.imagenet import download_class_names, find_class_by_text

classes = download_class_names(imagenet_dir)
find_class_by_text(classes, "military")

2024-06-28 16:25:43,190 - INFO - File imagenet/imagenet21k_wordnet_ids.txt already exists. Skipping download.
2024-06-28 16:25:43,191 - INFO - File imagenet/imagenet21k_wordnet_lemmas.txt already exists. Skipping download.


{'n03762982': 'military_hospital',
 'n03763727': 'military_quarters',
 'n03763968': 'military_uniform',
 'n03764276': 'military_vehicle',
 'n04552348': 'warplane, military_plane',
 'n08249459': 'concert_band, military_band',
 'n09809538': 'army_engineer, military_engineer',
 'n09943239': 'commissioned_military_officer',
 'n10316360': 'military_attache',
 'n10316527': 'military_chaplain, padre, Holy_Joe, sky_pilot',
 'n10316862': 'military_leader',
 'n10317007': 'military_officer, officer',
 'n10317500': 'military_policeman, MP',
 'n10512372': 'recruit, military_recruit',
 'n10582746': 'serviceman, military_man, man, military_personnel',
 'n10759331': 'volunteer, military_volunteer, voluntary'}

We can now download images and annotations for the relevant classes. The `download_imagenet_detections` function will download the images and annotations for the given class ids **if the annotations exist** (not all classes have been annotated).

In [4]:
from adomvi.datasets.imagenet import download_imagenet_detections

class_ids = ["n02740300", "n04389033", "n02740533", "n04464852", "n03764276"]
download_imagenet_detections(class_ids, imagenet_dir)

2024-06-28 16:25:43,244 - INFO - File imagenet/bboxes_annotations.tar.gz already exists. Skipping download.
2024-06-28 16:25:43,804 - INFO - There are not annotations for class n02740300.
2024-06-28 16:25:43,805 - INFO - Annotations directory imagenet/labels/n04389033 already exists. Skipping extract.
2024-06-28 16:25:43,805 - INFO - There are not annotations for class n02740533.
2024-06-28 16:25:43,806 - INFO - There are not annotations for class n04464852.
2024-06-28 16:25:43,806 - INFO - There are not annotations for class n03764276.
2024-06-28 16:25:43,807 - INFO - Deleting annotations dir.


The data we just downloaded into the `imagenet` directory is not all clean: there are annotations which have no corresponding image. We need to remove those labels, otherwise this causes errors when importing the data into fiftyone.

In [5]:
from adomvi.datasets.imagenet import cleanup_labels_without_images

cleanup_labels_without_images(imagenet_dir)

2024-06-28 16:25:43,867 - INFO - Deleting 0 labels without images


We can now create a new dataset with `fiftyone`. Fiftyone allows us to manage images annotated with bounding boxes and labels, to merge datasets from different sources, and to split the datasets and prepare them for processing.

In [6]:
import fiftyone as fo

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=imagenet_dir,
    dataset_type=fo.types.VOCDetectionDataset,
)

dataset.map_labels("ground_truth", {"n04389033": "AFV"}).save()

 100% |█████████████████| 378/378 [450.8ms elapsed, 0s remaining, 844.1 samples/s]      


Once our dataset is created, we can launch a session to display the dataset and view the annotated images

In [ ]:
session = fo.launch_app(dataset)

### Add OpenImage samples

The ImageNet dataset only contained 378 annotated images of tanks, so we'll look into other available datasets to improve training of the model. We’ll load [Open Images](https://storage.googleapis.com/openimages/web/index.html) samples with `Tank` detection labels, passing in `only_matching=True` to only load the `Tank` labels. We then map these labels by changing `Tank` into `AFV`.

In [7]:
import fiftyone.zoo as foz

oi_samples = foz.load_zoo_dataset(
    "open-images-v7",
    classes = ["Tank"],
    only_matching=True,
    label_types="detections"
).map_labels(
    "ground_truth",
    {"Tank":"AFV"}
)

Necessary images already downloaded
Existing download of split 'train' is sufficient
Necessary images already downloaded
Existing download of split 'test' is sufficient
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading 'open-images-v7' split 'train'
 100% |███████████████| 1062/1062 [1.7s elapsed, 0s remaining, 635.4 samples/s]         
Loading 'open-images-v7' split 'test'
 100% |█████████████████| 134/134 [226.4ms elapsed, 0s remaining, 601.6 samples/s]     
Loading 'open-images-v7' split 'validation'
 100% |███████████████████| 50/50 [69.8ms elapsed, 0s remaining, 716.0 samples/s]     
Dataset 'open-images-v7' created


We can add these new samples into our training dataset with `merge_samples()`:

In [8]:
dataset.merge_samples(oi_samples)

In [ ]:
session.show()

### Add Roboflow dataset

We now have 1624 annotated images of tanks, which is good but still a bit small. To get even more training images, we'll load another annotated dataset of military vehicles, made available by Tuomo Hiippala from the Digital Geography Lab, on [Roboflow](https://universe.roboflow.com/capstoneproject/russian-military-annotated).

The following code will download, restructure, and import the dataset into fityone.

In [9]:
from adomvi.datasets.roboflow import download_roboflow_dataset, restructure_dataset

roboflow_dir = Path() / "roboflow"
url="https://universe.roboflow.com/ds/P2jPq32qKU?key=E4MIo8mavP"
download_roboflow_dataset(url, roboflow_dir)
restructure_dataset(roboflow_dir)

2024-06-28 16:32:03,250 - INFO - Downloading roboflow/dataset_rf.zip from https://universe.roboflow.com/ds/P2jPq32qKU?key=E4MIo8mavP ...
2024-06-28 16:32:07,600 - INFO - Download complete.
2024-06-28 16:32:07,799 - INFO - Extracted to roboflow
2024-06-28 16:32:07,997 - INFO - Dataset dir restructured successfully.


Since this new dataset defines the type of vehicle as classes, we'll map those to our four classes.

In [10]:
# Import the roboflow dataset
dataset_rf = fo.Dataset.from_dir(
    dataset_dir=roboflow_dir,
    dataset_type=fo.types.VOCDetectionDataset,
)

# Define the existing labels and their mappings
label_mapping = {
    "bm-21": "AFV",
    "t-80": "AFV",
    "t-64": "AFV",
    "t-72": "AFV",
    "bmp-1": "AFV",
    "bmp-2": "AFV",
    "bmd-2": "AFV",
    "btr-70": "APC",
    "btr-80": "APC",
    "mt-lb": "APC",
}

# Map the labels
dataset_rf.map_labels(
    "ground_truth",
    label_mapping
).save()

 100% |███████████████| 1042/1042 [1.5s elapsed, 0s remaining, 677.6 samples/s]         


Optionally, we can remove the few images which have no labels and actually don't contain images of military vehicles.

In [11]:
# from adomvi.datasets.roboflow import delete_images_without_labels

# delete_images_without_labels(dataset_rf)

Finally, we can add these new samples into our training dataset.

In [12]:
dataset.merge_samples(dataset_rf)

In [ ]:
session.show()

### Add a custom dataset scraped from Google

We now have 2666 images of military vehicles. To improve that a bit, we provide a [sample dataset](https://github.com/jonasrenault/adomvi/releases/download/v1.2.0/military-vehicles-dataset.tar.gz) of images scraped from Google and annotated with our 4 classes (*AFV*, *APC*, *LAV* & *MEV*). Let's download the dataset and merge it with our existing dataset.

In [13]:
from adomvi.datasets.google import download_google_dataset

google_dir = Path() / "google"
url = "https://github.com/jonasrenault/adomvi/releases/download/v1.2.0/military-vehicles-dataset.tar.gz"

download_google_dataset(url, google_dir)

2024-06-28 16:32:09,877 - INFO - Downloading google/military-vehicles-dataset.tar.gz from https://github.com/jonasrenault/adomvi/releases/download/v1.2.0/military-vehicles-dataset.tar.gz ...
2024-06-28 16:32:18,724 - INFO - Download complete.
2024-06-28 16:32:19,243 - INFO - Extracted to google


In [14]:
dataset_google = fo.Dataset.from_dir(
    dataset_dir=google_dir / "dataset",
    dataset_type=fo.types.YOLOv4Dataset,
)

Images file '/home/jrenault/workspace/adomvi2/notebooks/google/dataset/images.txt' not found. Listing data directory '/home/jrenault/workspace/adomvi2/notebooks/google/dataset/data/' instead
 100% |█████████████████| 669/669 [589.4ms elapsed, 0s remaining, 1.1K samples/s]      


In [15]:
dataset.merge_samples(dataset_google)


In [ ]:
session.show()

### Export dataset to disk

Now that our dataset is created, we'll export it into a format supported by YOLOv8 to train our model.

We first remove tags from the dataset, and split it into a train, val and test sets.

In [16]:
import fiftyone.utils.random as four

## delete existing tags to start fresh
dataset.untag_samples(dataset.distinct("tags"))

## split into train, test and val
four.random_split(dataset, {"train": 0.8, "val": 0.1, "test": 0.1})

Once our dataset is split, we can export it to a specific directory.

In [17]:
from adomvi.yolo.utils import export_yolo_data

export_dir = Path() / "dataset"
export_yolo_data(dataset, export_dir, ["AFV", "APC", "MEV", "LAV"], split = ["train", "val", "test"], overwrite=True)

 100% |███████████████| 2668/2668 [3.6s elapsed, 0s remaining, 1.4K samples/s]       
Directory 'dataset' already exists; export will be merged with existing files
 100% |█████████████████| 334/334 [456.7ms elapsed, 0s remaining, 731.4 samples/s]      
Directory 'dataset' already exists; export will be merged with existing files
 100% |█████████████████| 333/333 [444.5ms elapsed, 0s remaining, 749.2 samples/s]      
